# Prerocessing Data and EDA

In [7]:
# Imports
import pandas as pd
import numpy as np
from IPython.display import display, Markdown

# Set pandas options
pd.set_option('display.max_columns', None)

# Load datasets
df_ksi = pd.read_csv("Motor Vehicle Collisions with KSI Data - 4326.csv")
df_hourly = pd.read_csv("hourly_final.csv")
df_tmc = pd.read_csv("tmc_raw_data_2010_2019.csv")
df_collisions = pd.read_csv("Traffic_Collisions_Toronto_data.csv")

# Define a function to display a title and first few rows
def display_table(title, df):
    display(Markdown(f"### {title}"))
    display(df.head())

# Display each dataset
display_table("KSI Dataset", df_ksi)
display_table("Env Canada Dataset", df_hourly)
display_table("TMC Dataset (2010–2019)", df_tmc)
display_table("Toronto Police Collision Dataset", df_collisions)


C:\Users\manav\AppData\Local\Temp\ipykernel_14124\1006396101.py:11: DtypeWarning: Columns (4,8,17,19,20,24,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df_hourly = pd.read_csv("hourly_final.csv")


### KSI Dataset

,_id,ACCNUM,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,ACCLOC,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,FATAL_NO,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,geometry
0,1,893184.0,2006-01-01,236,WOODBINE AVE,O CONNOR DR,NaN,Major Arterial,Toronto and East York,Intersection Related,No Control,Clear,Dark,Wet,Non-Fatal Injury,Approaching,Passenger,50 to 54,Major,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,Yes,Yes,Yes,NaN,Yes,NaN,60,Woodbine-Lumsden,60,Woodbine-Lumsden (60),D55,"{""coordinates"": [[-79.318797000266, 43.6995950..."
1,2,893184.0,2006-01-01,236,WOODBINE AVE,O CONNOR DR,NaN,Major Arterial,Toronto and East York,Intersection Related,No Control,Clear,Dark,Wet,Non-Fatal Injury,Approaching,Passenger,15 to 19,Minor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,Yes,Yes,Yes,NaN,Yes,NaN,60,Woodbine-Lumsden,60,Woodbine-Lumsden (60),D55,"{""coordinates"": [[-79.318797000266, 43.6995950..."
2,3,893184.0,2006-01-01,236,WOODBINE AVE,O CONNOR DR,NaN,Major Arterial,Toronto and East York,Intersection Related,No Control,Clear,Dark,Wet,Non-Fatal Injury,Approaching,Driver,55 to 59,Minor,NaN,North,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,Yes,Yes,Yes,NaN,Yes,NaN,60,Woodbine-Lumsden,60,Woodbine-Lumsden (60),D55,"{""coordinates"": [[-79.318797000266, 43.6995950..."
3,4,893184.0,2006-01-01,236,WOODBINE AVE,O CONNOR DR,NaN,Major Arterial,Toronto and East York,Intersection Related,No Control,Clear,Dark,Wet,Non-Fatal Injury,Approaching,Passenger,20 to 24,Minor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,Yes,Yes,Yes,NaN,Yes,NaN,60,Woodbine-Lumsden,60,Woodbine-Lumsden (60),D55,"{""coordinates"": [[-79.318797000266, 43.6995950..."
4,5,893184.0,2006-01-01,236,WOODBINE AVE,O CONNOR DR,NaN,Major Arterial,Toronto and East York,Intersection Related,No Control,Clear,Dark,Wet,Non-Fatal Injury,Approaching,Passenger,15 to 19,Minor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,Yes,Yes,Yes,NaN,Yes,NaN,60,Woodbine-Lumsden,60,Woodbine-Lumsden (60),D55,"{""coordinates"": [[-79.318797000266, 43.6995950..."


### Env Canada Dataset

,x,y,LOCAL_DATE,STATION_PRESSURE,TEMP_FLAG,WINDCHILL,LOCAL_HOUR,RELATIVE_HUMIDITY,WIND_DIRECTION_FLAG,WIND_DIRECTION,HUMIDEX_FLAG,WEATHER_FRE_DESC,LOCAL_YEAR,UTC_MONTH,WEATHER_ENG_DESC,STATION_NAME,UTC_DAY,STATION_PRESSURE_FLAG,ID,PRECIP_AMOUNT_FLAG,WIND_SPEED_FLAG,PRECIP_AMOUNT,WIND_SPEED,UTC_YEAR,DEW_POINT_TEMP_FLAG,LOCAL_MONTH,CLIMATE_IDENTIFIER,LOCAL_DAY,PROVINCE_CODE,UTC_DATE,DEW_POINT_TEMP,TEMP,WINDCHILL_FLAG,VISIBILITY,RELATIVE_HUMIDITY_FLAG,HUMIDEX,VISIBILITY_FLAG
0,-79.4,43.666667,2025-03-12 07:00:00,100.47,NaN,NaN,7,54.0,NaN,NaN,NaN,ND,2025,3,NaN,TORONTO CITY,12,NaN,6158355.2025.3.12.7,NaN,NaN,0.0,NaN,2025,NaN,3,6158355,12,ON,2025-03-12T12:00:00,-9.9,-1.8,NaN,NaN,NaN,NaN,NaN
1,-79.4,43.666667,2025-03-12 06:00:00,100.39,NaN,NaN,6,53.0,NaN,NaN,NaN,ND,2025,3,NaN,TORONTO CITY,12,NaN,6158355.2025.3.12.6,NaN,NaN,0.0,NaN,2025,NaN,3,6158355,12,ON,2025-03-12T11:00:00,-10.2,-1.9,NaN,NaN,NaN,NaN,NaN
2,-79.4,43.666667,2025-03-12 05:00:00,100.34,NaN,NaN,5,46.0,NaN,NaN,NaN,ND,2025,3,NaN,TORONTO CITY,12,NaN,6158355.2025.3.12.5,NaN,NaN,0.0,NaN,2025,NaN,3,6158355,12,ON,2025-03-12T10:00:00,-12.0,-1.8,NaN,NaN,NaN,NaN,NaN
3,-79.4,43.666667,2025-03-12 04:00:00,100.29,NaN,NaN,4,47.0,NaN,NaN,NaN,ND,2025,3,NaN,TORONTO CITY,12,NaN,6158355.2025.3.12.4,NaN,NaN,0.0,NaN,2025,NaN,3,6158355,12,ON,2025-03-12T09:00:00,-11.5,-1.7,NaN,NaN,NaN,NaN,NaN
4,-79.4,43.666667,2025-03-12 03:00:00,100.30,NaN,NaN,3,48.0,NaN,NaN,NaN,ND,2025,3,NaN,TORONTO CITY,12,NaN,6158355.2025.3.12.3,NaN,NaN,0.0,NaN,2025,NaN,3,6158355,12,ON,2025-03-12T08:00:00,-10.9,-1.4,NaN,NaN,NaN,NaN,NaN


### TMC Dataset (2010–2019)

,_id,count_id,count_date,location_name,longitude,latitude,centreline_type,centreline_id,px,start_time,end_time,n_appr_cars_r,n_appr_cars_t,n_appr_cars_l,s_appr_cars_r,s_appr_cars_t,s_appr_cars_l,e_appr_cars_r,e_appr_cars_t,e_appr_cars_l,w_appr_cars_r,w_appr_cars_t,w_appr_cars_l,n_appr_truck_r,n_appr_truck_t,n_appr_truck_l,s_appr_truck_r,s_appr_truck_t,s_appr_truck_l,e_appr_truck_r,e_appr_truck_t,e_appr_truck_l,w_appr_truck_r,w_appr_truck_t,w_appr_truck_l,n_appr_bus_r,n_appr_bus_t,n_appr_bus_l,s_appr_bus_r,s_appr_bus_t,s_appr_bus_l,e_appr_bus_r,e_appr_bus_t,e_appr_bus_l,w_appr_bus_r,w_appr_bus_t,w_appr_bus_l,n_appr_peds,s_appr_peds,e_appr_peds,w_appr_peds,n_appr_bike,s_appr_bike,e_appr_bike,w_appr_bike
0,1,25141,2010-01-14,Lobb Ave / Shaw St,-79.417131,43.646929,2,13467329,1970.0,2010-01-14T08:30:00,2010-01-14T08:45:00,0,62,5,0,54,0,4,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,1,4,0,0,0,0,0
1,2,25141,2010-01-14,Lobb Ave / Shaw St,-79.417131,43.646929,2,13467329,1970.0,2010-01-14T09:00:00,2010-01-14T09:15:00,0,58,4,0,40,0,5,0,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,9,4,0,0,0,0,0
2,3,25099,2010-01-11,Richmond St W / Brant St,-79.398657,43.647474,2,13467191,2023.0,2010-01-11T13:15:00,2010-01-11T13:30:00,0,0,0,0,0,13,0,110,17,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,3,3,3,9,0,0,0,0
3,4,25099,2010-01-11,Richmond St W / Brant St,-79.398657,43.647474,2,13467191,2023.0,2010-01-11T13:30:00,2010-01-11T13:45:00,0,0,0,0,0,17,0,107,12,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,6,1,12,0,0,0,0
4,5,25099,2010-01-11,Richmond St W / Brant St,-79.398657,43.647474,2,13467191,2023.0,2010-01-11T13:45:00,2010-01-11T14:00:00,0,0,0,0,0,17,0,111,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,3,6,4,12,0,0,0,0


### Toronto Police Collision Dataset

,X,Y,OBJECTID,EventUniqueId,OccurrenceDate,Month,Day_of_Week,Year,Hour,Division,Atom,Neighbourhood,Fatalities,Injury_Collisions,FTR_Collisions,PD_Collisions,Longitude,Latitude,ObjectId2
0,0.0,0.0,1,GO-20141001,2014/02/07 05:00:00+00,February,Friday,2014,16,NSA,NSA,NSA,0,NO,NO,YES,0.0,0.0,1
1,0.0,0.0,2,GO-20141225593,2014/01/02 05:00:00+00,January,Thursday,2014,3,NSA,NSA,NSA,0,NO,YES,NO,0.0,0.0,2
2,0.0,0.0,3,GO-20141260499,2014/01/01 05:00:00+00,January,Wednesday,2014,2,NSA,NSA,NSA,0,YES,NO,NO,0.0,0.0,3
3,0.0,0.0,4,GO-20141260663,2014/01/01 05:00:00+00,January,Wednesday,2014,3,NSA,NSA,NSA,0,NO,NO,YES,0.0,0.0,4
4,0.0,0.0,5,GO-20141261162,2014/01/01 05:00:00+00,January,Wednesday,2014,5,NSA,NSA,NSA,0,YES,NO,NO,0.0,0.0,5


In [8]:
# Define start and end range
start_date = pd.to_datetime('2015-01-01')
end_date = pd.to_datetime('2020-12-31')

# Parse datetime columns for each dataset
df_ksi['DATE'] = pd.to_datetime(df_ksi['DATE'], errors='coerce')
df_hourly['LOCAL_DATE'] = pd.to_datetime(df_hourly['LOCAL_DATE'], errors='coerce')
df_tmc['count_date'] = pd.to_datetime(df_tmc['count_date'], errors='coerce')

# Handle timezone-aware datetime in df_collisions
df_collisions['OccurrenceDate'] = pd.to_datetime(df_collisions['OccurrenceDate'], errors='coerce')
df_collisions['OccurrenceDate'] = df_collisions['OccurrenceDate'].dt.tz_localize(None)

# Filter all datasets by the 2015–2020 range
df_ksi_filtered = df_ksi[(df_ksi['DATE'] >= start_date) & (df_ksi['DATE'] <= end_date)]
df_hourly_filtered = df_hourly[(df_hourly['LOCAL_DATE'] >= start_date) & (df_hourly['LOCAL_DATE'] <= end_date)]
df_tmc_filtered = df_tmc[(df_tmc['count_date'] >= start_date) & (df_tmc['count_date'] <= end_date)]
df_collisions_filtered = df_collisions[(df_collisions['OccurrenceDate'] >= start_date) & (df_collisions['OccurrenceDate'] <= end_date)]

# Display filtered datasets
display_table("Filtered KSI Dataset (2015–2020)", df_ksi_filtered)
display_table("Filtered Environment Canada Weather Dataset (2015–2020)", df_hourly_filtered)
display_table("Filtered TMC Dataset (2015–2020)", df_tmc_filtered)
display_table("Filtered Toronto Police Collision Dataset (2015–2020)", df_collisions_filtered)


### Filtered KSI Dataset (2015–2020)

,_id,ACCNUM,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,ACCLOC,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,FATAL_NO,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,geometry
11303,11304,NaN,2015-01-01,624,KEELE ST,COLVILLE RD,NaN,Major Arterial,Etobicoke York,At Intersection,No Control,Clear,"Dawn, artificial",Dry,Non-Fatal Injury,Approaching,Driver,20 to 24,Major,NaN,South,"Automobile, Station Wagon",Going Ahead,Lost control,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,Yes,Yes,Yes,NaN,NaN,NaN,30,Brookhaven-Amesbury,30,Brookhaven-Amesbury (30),D12,"{""coordinates"": [[-79.4774959999594, 43.706175..."
11304,11305,NaN,2015-01-01,624,KEELE ST,COLVILLE RD,NaN,Major Arterial,Etobicoke York,At Intersection,No Control,Clear,"Dawn, artificial",Dry,Non-Fatal Injury,Approaching,Driver,30 to 34,Minor,NaN,North,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,Yes,Yes,Yes,NaN,NaN,NaN,30,Brookhaven-Amesbury,30,Brookhaven-Amesbury (30),D12,"{""coordinates"": [[-79.4774959999594, 43.706175..."
11305,11306,NaN,2015-01-01,624,KEELE ST,COLVILLE RD,NaN,Major Arterial,Etobicoke York,At Intersection,No Control,Clear,"Dawn, artificial",Dry,Non-Fatal Injury,Approaching,Passenger,20 to 24,Minor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,Yes,Yes,Yes,NaN,NaN,NaN,30,Brookhaven-Amesbury,30,Brookhaven-Amesbury (30),D12,"{""coordinates"": [[-79.4774959999594, 43.706175..."
11306,11307,NaN,2015-01-02,949,2211 ST CLAIR AVE W,NaN,23 m East of,Major Arterial,Etobicoke York,Non Intersection,No Control,Clear,Daylight,Dry,Non-Fatal Injury,Turning Movement,Driver,60 to 64,Major,NaN,West,"Automobile, Station Wagon",Making U Turn,Improper Turn,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90,Junction Area,90,Junction Area (90),D11,"{""coordinates"": [[-79.4751849996918, 43.670848..."
11307,11308,NaN,2015-01-02,949,2211 ST CLAIR AVE W,NaN,23 m East of,Major Arterial,Etobicoke York,Non Intersection,No Control,Clear,Daylight,Dry,Non-Fatal Injury,Turning Movement,Truck Driver,30 to 34,NaN,NaN,West,"Truck - Closed (Blazer, etc)",Going Ahead,Driving Properly,Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90,Junction Area,90,Junction Area (90),D11,"{""coordinates"": [[-79.4751849996918, 43.670848..."


### Filtered Environment Canada Weather Dataset (2015–2020)

,x,y,LOCAL_DATE,STATION_PRESSURE,TEMP_FLAG,WINDCHILL,LOCAL_HOUR,RELATIVE_HUMIDITY,WIND_DIRECTION_FLAG,WIND_DIRECTION,HUMIDEX_FLAG,WEATHER_FRE_DESC,LOCAL_YEAR,UTC_MONTH,WEATHER_ENG_DESC,STATION_NAME,UTC_DAY,STATION_PRESSURE_FLAG,ID,PRECIP_AMOUNT_FLAG,WIND_SPEED_FLAG,PRECIP_AMOUNT,WIND_SPEED,UTC_YEAR,DEW_POINT_TEMP_FLAG,LOCAL_MONTH,CLIMATE_IDENTIFIER,LOCAL_DAY,PROVINCE_CODE,UTC_DATE,DEW_POINT_TEMP,TEMP,WINDCHILL_FLAG,VISIBILITY,RELATIVE_HUMIDITY_FLAG,HUMIDEX,VISIBILITY_FLAG
36750,-79.4,43.666667,2020-12-31 00:00:00,99.60,NaN,NaN,0,63.0,NaN,NaN,NaN,ND,2020,12,NaN,TORONTO CITY,31,NaN,6158355.2020.12.31.0,NaN,NaN,0.0,NaN,2020,NaN,12,6158355,31,ON,2020-12-31T05:00:00,-3.2,3.2,NaN,NaN,NaN,NaN,NaN
36751,-79.4,43.666667,2020-12-30 23:00:00,99.51,NaN,NaN,23,66.0,NaN,NaN,NaN,ND,2020,12,NaN,TORONTO CITY,31,NaN,6158355.2020.12.30.23,NaN,NaN,0.0,NaN,2020,NaN,12,6158355,30,ON,2020-12-31T04:00:00,-2.3,3.5,NaN,NaN,NaN,NaN,NaN
36752,-79.4,43.666667,2020-12-30 22:00:00,99.43,NaN,NaN,22,70.0,NaN,NaN,NaN,ND,2020,12,NaN,TORONTO CITY,31,NaN,6158355.2020.12.30.22,NaN,NaN,0.0,NaN,2020,NaN,12,6158355,30,ON,2020-12-31T03:00:00,-0.9,4.1,NaN,NaN,NaN,NaN,NaN
36753,-79.4,43.666667,2020-12-30 21:00:00,99.33,NaN,NaN,21,79.0,NaN,NaN,NaN,ND,2020,12,NaN,TORONTO CITY,31,NaN,6158355.2020.12.30.21,NaN,NaN,0.0,NaN,2020,NaN,12,6158355,30,ON,2020-12-31T02:00:00,2.0,5.3,NaN,NaN,NaN,NaN,NaN
36754,-79.4,43.666667,2020-12-30 20:00:00,99.28,NaN,NaN,20,86.0,NaN,NaN,NaN,ND,2020,12,NaN,TORONTO CITY,31,NaN,6158355.2020.12.30.20,NaN,NaN,0.1,NaN,2020,NaN,12,6158355,30,ON,2020-12-31T01:00:00,3.5,5.6,NaN,NaN,NaN,NaN,NaN


### Filtered TMC Dataset (2015–2020)

,_id,count_id,count_date,location_name,longitude,latitude,centreline_type,centreline_id,px,start_time,end_time,n_appr_cars_r,n_appr_cars_t,n_appr_cars_l,s_appr_cars_r,s_appr_cars_t,s_appr_cars_l,e_appr_cars_r,e_appr_cars_t,e_appr_cars_l,w_appr_cars_r,w_appr_cars_t,w_appr_cars_l,n_appr_truck_r,n_appr_truck_t,n_appr_truck_l,s_appr_truck_r,s_appr_truck_t,s_appr_truck_l,e_appr_truck_r,e_appr_truck_t,e_appr_truck_l,w_appr_truck_r,w_appr_truck_t,w_appr_truck_l,n_appr_bus_r,n_appr_bus_t,n_appr_bus_l,s_appr_bus_r,s_appr_bus_t,s_appr_bus_l,e_appr_bus_r,e_appr_bus_t,e_appr_bus_l,w_appr_bus_r,w_appr_bus_t,w_appr_bus_l,n_appr_peds,s_appr_peds,e_appr_peds,w_appr_peds,n_appr_bike,s_appr_bike,e_appr_bike,w_appr_bike
99531,99532,31741,2015-01-06,Royal York Rd / Newcastle St,-79.49892,43.617497,2,13469941,1818.0,2015-01-06T07:30:00,2015-01-06T07:45:00,2,48,6,25,115,4,5,0,6,1,6,0,0,8,2,2,15,0,2,0,1,0,0,0,0,4,0,0,9,0,0,0,1,0,0,0,3,22,0,12,0,1,0,0
99532,99533,31741,2015-01-06,Royal York Rd / Newcastle St,-79.49892,43.617497,2,13469941,1818.0,2015-01-06T07:45:00,2015-01-06T08:00:00,1,48,5,10,98,2,7,1,5,1,8,3,0,12,0,1,7,0,1,0,2,0,2,0,0,6,1,0,9,0,0,0,0,0,0,0,6,10,4,0,0,0,0,0
99533,99534,31741,2015-01-06,Royal York Rd / Newcastle St,-79.49892,43.617497,2,13469941,1818.0,2015-01-06T08:00:00,2015-01-06T08:15:00,1,63,6,30,119,5,9,2,17,0,10,4,0,14,2,0,9,0,1,0,2,0,0,0,0,5,0,0,4,0,0,0,2,0,0,1,7,25,11,16,2,1,0,0
99534,99535,31741,2015-01-06,Royal York Rd / Newcastle St,-79.49892,43.617497,2,13469941,1818.0,2015-01-06T08:15:00,2015-01-06T08:30:00,2,73,4,6,117,3,7,1,18,3,2,1,1,11,1,1,16,1,2,0,0,2,0,0,0,6,0,0,6,0,0,0,1,0,1,0,4,4,6,11,0,0,0,0
99535,99536,31741,2015-01-06,Royal York Rd / Newcastle St,-79.49892,43.617497,2,13469941,1818.0,2015-01-06T08:30:00,2015-01-06T08:45:00,1,78,11,12,136,4,5,2,28,5,7,5,3,8,2,1,11,0,0,0,1,0,0,0,0,3,0,0,8,0,0,0,2,0,0,0,7,7,4,3,0,0,0,0


### Filtered Toronto Police Collision Dataset (2015–2020)

,X,Y,OBJECTID,EventUniqueId,OccurrenceDate,Month,Day_of_Week,Year,Hour,Division,Atom,Neighbourhood,Fatalities,Injury_Collisions,FTR_Collisions,PD_Collisions,Longitude,Latitude,ObjectId2
9000,0.0,0.0,9701,GO-2015160241,2015-01-28 05:00:00,January,Wednesday,2015,0,NSA,NSA,NSA,0,YES,NO,NO,0.0,0.0,9001
9001,0.0,0.0,9702,GO-20151603134,2015-09-16 04:00:00,September,Wednesday,2015,17,NSA,NSA,NSA,0,YES,NO,NO,0.0,0.0,9002
9002,0.0,0.0,9703,GO-2015160956,2015-01-28 05:00:00,January,Wednesday,2015,6,D41,NSA,NSA,1,NO,NO,NO,0.0,0.0,9003
9003,0.0,0.0,9704,GO-20151614143,2015-09-18 04:00:00,September,Friday,2015,11,NSA,NSA,NSA,0,YES,NO,NO,0.0,0.0,9004
9004,0.0,0.0,9705,GO-20151614153,2015-09-18 04:00:00,September,Friday,2015,11,NSA,NSA,NSA,0,YES,NO,NO,0.0,0.0,9005


In [11]:
# Extract and format unique KSI street pairs
ksi_streets = df_ksi_filtered[['STREET1', 'STREET2']].dropna().astype(str)
ksi_streets['combined'] = ksi_streets['STREET1'].str.upper().str.strip() + ' / ' + ksi_streets['STREET2'].str.upper().str.strip()
unique_ksi_pairs = ksi_streets['combined'].drop_duplicates().sort_values().reset_index(drop=True)

display(Markdown("### Unique KSI Street Pairs (Combined)"))
display(unique_ksi_pairs.to_frame(name='Street Pair'))

# Extract unique location names from TMC dataset
tmc_locations = df_tmc_filtered['location_name'].dropna().str.upper().str.strip().drop_duplicates().sort_values().reset_index(drop=True)

display(Markdown("### Unique TMC Locations"))
display(tmc_locations.to_frame(name='Location Name'))

# Extract unique neighbourhoods from Toronto Police Collision dataset
collision_neighborhoods = df_collisions_filtered['Neighbourhood'].dropna().str.upper().str.strip().drop_duplicates().sort_values().reset_index(drop=True)

display(Markdown("### Unique Toronto Police Collision Neighbourhoods"))
display(collision_neighborhoods.to_frame(name='Neighbourhood'))



### Unique KSI Street Pairs (Combined)

,Street Pair
0,116 ELINOR AVE / JOY DR
1,1595 BATHURST ST / LONSMOUNT DR
2,2301 BRIMLEY RD / HUNTINGWOOD DR
3,259 WELLESLEY ST E / ROSE AVE
4,4 RIDGEMOUNT RD / CHAPMAN RD
...,...
1652,YONGE ST / YORK MILLS RD
1653,YORK MILLS RD / DON VALLEY PARKWAY S
1654,YORK MILLS RD / SCARSDALE RD
1655,YORK MILLS RD / YONGE ST


### Unique TMC Locations

,Location Name
0,404 S FAIRVIEW MALL RAMP / FAIRVIEW MALL 404 S...
1,427 N FASKEN RAMP / FASKEN DR
2,ACACIA RD / DAVISVILLE AVE
3,ADELAIDE ST E / BERKELEY ST
4,ADELAIDE ST E / CHURCH ST
...,...
2798,YORK ST / BREMNER BLVD
2799,YORK ST / WELLINGTON ST W
2800,YORKDALE RD / 401 W ALLEN YORKDALE RAMP
2801,YORKDALE RD / ALLEN N YORKDALE RD RAMP / YORKD...


### Unique Toronto Police Collision Neighbourhoods

,Neighbourhood
0,AGINCOURT NORTH (129)
1,AGINCOURT SOUTH-MALVERN WEST (128)
2,ALDERWOOD (20)
3,ANNEX (95)
4,BANBURY-DON MILLS (42)
...,...
136,WYCHWOOD (94)
137,YONGE-EGLINTON (100)
138,YONGE-ST.CLAIR (97)
139,YORK UNIVERSITY HEIGHTS (27)


In [ ]:
import re
from collections import defaultdict

# Normalize both KSI and TMC names (strip, uppercase)
normalized_ksi = [s.upper().strip() for s in unique_ksi_pairs]
normalized_tmc = [s.upper().strip() for s in tmc_locations]

# Extract individual street components from KSI
def split_streets(street_pair):
    streets = [s.strip() for s in re.split(r'/|&', street_pair)]
    return streets if len(streets) == 2 else [streets[0], ""]

# Match tracking
matches = []
no_matches = []

for ksi_pair in normalized_ksi:
    streets = split_streets(ksi_pair)
    s1, s2 = streets[0], streets[1]

    # Build all forms to check
    patterns_to_match = {
        f"{s1} / {s2}",
        f"{s2} / {s1}",
        s1,
        s2
    }

    found_match = False
    for tmc in normalized_tmc:
        for pattern in patterns_to_match:
            if pattern and pattern in tmc:
                matches.append((ksi_pair, tmc))
                found_match = True
                break
        if found_match:
            break

    if not found_match:
        no_matches.append(ksi_pair)



In [13]:
# Display matches
display(Markdown(f"### ✅ Total Matches Found: {len(matches)}"))
display(pd.DataFrame(matches, columns=["KSI Pair", "Matched TMC Location"]))

# Display unmatched KSI locations
display(Markdown(f"### ❌ Unmatched KSI Pairs: {len(no_matches)}"))
display(pd.DataFrame(no_matches, columns=["KSI Pair (Unmatched)"]))


### ✅ Total Matches Found: 1542

,KSI Pair,Matched TMC Location
0,2301 BRIMLEY RD / HUNTINGWOOD DR,BIRCHMOUNT RD / HUNTINGWOOD DR
1,4 RIDGEMOUNT RD / CHAPMAN RD,SCARLETT RD / CHAPMAN RD
2,401 C E VICTORIA PR N RAMP / VICTORIA PARK AVE,DANFORTH AVE / VICTORIA PARK AVE
3,401 C E WESTON RD RAMP / WESTON RD,ALBION RD / WESTON RD ALBION RD RAMP / HUMBER ...
4,401 C W MCCOWAN N RAMP RD / MCCOWAN RD,EGLINTON AVE E / MCCOWAN RD
...,...,...
1537,YONGE ST / YORK MILLS RD,ADELAIDE ST E / YONGE ST / ADELAIDE ST W
1538,YORK MILLS RD / DON VALLEY PARKWAY S,VICTORIA PARK AVE / YORK MILLS RD
1539,YORK MILLS RD / SCARSDALE RD,VICTORIA PARK AVE / YORK MILLS RD
1540,YORK MILLS RD / YONGE ST,ADELAIDE ST E / YONGE ST / ADELAIDE ST W


### ❌ Unmatched KSI Pairs: 115

,KSI Pair (Unmatched)
0,116 ELINOR AVE / JOY DR
1,1595 BATHURST ST / LONSMOUNT DR
2,259 WELLESLEY ST E / ROSE AVE
3,4 THE DONWAY E / GUYTOI CRT
4,4400 DUFFERIN ST / STANSTEAD DR
...,...
110,WARDEN AVE / BAMBURGH CRCL
111,WILSON AVE / AVENUE RD
112,YONGE ST / ALBERTUS AVE
113,YONGE ST / ELM ST


In [16]:
# Convert matches list to DataFrame
matched_df = pd.DataFrame(matches, columns=["KSI Pair", "Matched TMC Location"])

# Keep only matched KSI pairs
matched_ksi_pairs = set(matched_df["KSI Pair"])
df_ksi_matched = ksi_streets[ksi_streets['combined'].isin(matched_ksi_pairs)].copy()

# Keep only matched TMC locations
matched_tmc_locs = set(matched_df["Matched TMC Location"])
df_tmc_matched = df_tmc_filtered[
    df_tmc_filtered['location_name'].str.upper().str.strip().isin(matched_tmc_locs)
].copy()

# Show the final filtered versions
display(Markdown("### ✅ Filtered KSI Dataset (Only Matched Rows)"))
display(df_ksi_matched)

display(Markdown("### ✅ Filtered TMC Dataset (Only Matched Rows)"))
display(df_tmc_matched)


### ✅ Filtered KSI Dataset (Only Matched Rows)

,STREET1,STREET2,combined
11303,KEELE ST,COLVILLE RD,KEELE ST / COLVILLE RD
11304,KEELE ST,COLVILLE RD,KEELE ST / COLVILLE RD
11305,KEELE ST,COLVILLE RD,KEELE ST / COLVILLE RD
11308,BAYVIEW AVE,SHEPPARD AVE E,BAYVIEW AVE / SHEPPARD AVE E
11309,BAYVIEW AVE,SHEPPARD AVE E,BAYVIEW AVE / SHEPPARD AVE E
...,...,...,...
16869,LAWRENCE AVE E,KENNEDY RD,LAWRENCE AVE E / KENNEDY RD
16870,DUNDAS ST E,OSKENONTON LANE,DUNDAS ST E / OSKENONTON LANE
16871,DUNDAS ST E,OSKENONTON LANE,DUNDAS ST E / OSKENONTON LANE
16872,KENNEDY RD,SHEPPARD AVE E,KENNEDY RD / SHEPPARD AVE E


### ✅ Filtered TMC Dataset (Only Matched Rows)

,_id,count_id,count_date,location_name,longitude,latitude,centreline_type,centreline_id,px,start_time,end_time,n_appr_cars_r,n_appr_cars_t,n_appr_cars_l,s_appr_cars_r,s_appr_cars_t,s_appr_cars_l,e_appr_cars_r,e_appr_cars_t,e_appr_cars_l,w_appr_cars_r,w_appr_cars_t,w_appr_cars_l,n_appr_truck_r,n_appr_truck_t,n_appr_truck_l,s_appr_truck_r,s_appr_truck_t,s_appr_truck_l,e_appr_truck_r,e_appr_truck_t,e_appr_truck_l,w_appr_truck_r,w_appr_truck_t,w_appr_truck_l,n_appr_bus_r,n_appr_bus_t,n_appr_bus_l,s_appr_bus_r,s_appr_bus_t,s_appr_bus_l,e_appr_bus_r,e_appr_bus_t,e_appr_bus_l,w_appr_bus_r,w_appr_bus_t,w_appr_bus_l,n_appr_peds,s_appr_peds,e_appr_peds,w_appr_peds,n_appr_bike,s_appr_bike,e_appr_bike,w_appr_bike
99659,99660,31749,2015-01-15,Bayview Ave / Byng Ave,-79.391495,43.781932,2,13445813,NaN,2015-01-15T07:30:00,2015-01-15T07:45:00,6,304,0,0,159,1,0,0,0,2,0,1,0,5,0,0,3,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
99660,99661,31749,2015-01-15,Bayview Ave / Byng Ave,-79.391495,43.781932,2,13445813,NaN,2015-01-15T07:45:00,2015-01-15T08:00:00,8,290,0,0,214,3,0,0,0,5,0,3,0,3,0,0,9,0,0,0,0,0,0,1,0,2,0,0,1,1,0,0,0,0,0,0,0,0,0,3,0,0,0,0
99661,99662,31749,2015-01-15,Bayview Ave / Byng Ave,-79.391495,43.781932,2,13445813,NaN,2015-01-15T08:00:00,2015-01-15T08:15:00,9,306,0,0,250,1,0,0,0,8,0,4,0,8,0,0,3,0,0,0,0,0,0,0,0,2,0,0,2,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
99662,99663,31749,2015-01-15,Bayview Ave / Byng Ave,-79.391495,43.781932,2,13445813,NaN,2015-01-15T08:15:00,2015-01-15T08:30:00,12,242,0,0,226,10,0,0,0,9,0,2,0,7,0,0,8,1,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0
99663,99664,31749,2015-01-15,Bayview Ave / Byng Ave,-79.391495,43.781932,2,13445813,NaN,2015-01-15T08:30:00,2015-01-15T08:45:00,18,271,0,0,220,9,0,0,0,14,0,3,0,6,0,0,12,1,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223504,223505,40229,2019-12-18,White Heather Blvd / McNicoll Ave,-79.267385,43.818954,2,13441995,NaN,2019-12-18T16:45:00,2019-12-18T17:00:00,0,0,0,8,0,3,0,136,11,15,160,0,0,0,0,8,0,0,0,3,0,0,3,0,0,0,0,0,0,0,0,1,0,0,2,0,0,2,0,0,0,0,0,0
223505,223506,40229,2019-12-18,White Heather Blvd / McNicoll Ave,-79.267385,43.818954,2,13441995,NaN,2019-12-18T17:00:00,2019-12-18T17:15:00,0,0,0,13,0,3,0,160,11,14,171,0,0,0,0,13,0,0,0,8,0,0,4,0,0,0,0,0,0,0,0,4,0,0,1,0,0,0,0,0,0,0,0,0
223506,223507,40229,2019-12-18,White Heather Blvd / McNicoll Ave,-79.267385,43.818954,2,13441995,NaN,2019-12-18T17:15:00,2019-12-18T17:30:00,0,0,0,11,0,6,0,146,7,13,163,0,0,0,0,11,0,0,0,2,0,0,2,0,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0
223507,223508,40229,2019-12-18,White Heather Blvd / McNicoll Ave,-79.267385,43.818954,2,13441995,NaN,2019-12-18T17:30:00,2019-12-18T17:45:00,0,0,0,7,0,9,0,154,18,8,193,0,0,0,0,7,0,0,0,3,0,0,7,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
